In [158]:
import numpy as np

from scipy import optimize

from types import SimpleNamespace

import math

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

In [155]:
def griewank(x):
    return griewank_(x[0],x[1])
    
def griewank_(x1,x2):
    A = x1**2/4000 + x2**2/4000
    B = np.cos(x1/np.sqrt(1))*np.cos(x2/np.sqrt(2))
    return A-B+1

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [156]:
# Parameters
par = SimpleNamespace()

par.tolerance = 10**(-8)

par.warmup_iterations = 10

par.maximum_iterations = 1000

In [159]:
def solve(par):
    
    initial_guess = np.empty(1000)
    counter_k = np.empty(1000)
    minimum = np.zeros(1000)


    for k in range(par.maximum_iterations):
        x1 = np.random.uniform(-600, 600)
        x2 = np.random.uniform(-600, 600)
                               
        if k < par.warmup_iterations:
            result = optimize.minimize(griewank_, x1, x2, method='BFGS')
        else:
            chi = 0.50 * (2 / (1+math.exp((k-par.warmup_iterations) /100)))
            x01 = chi*x1 - (1-chi) * 0
            x02 = chi*x2 - (1-chi) * 0
            result = optimize.minimize(griewank_, x01, x02, method='BFGS')
            if k == 0:
                minimum[k] = j
            if griewank_(x01,x02) < griewank_(x1,x2):
                j

        if result < par.tolerance:
            initial_guess[k] = result.fun
            counter_k[k] = result.x[0]
            
    return initial_guess, counter_k

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?